(tutorial-dynmat)=

# Dynamical Matrix Tutorial

In this tutorial you will learn how to use the main workflow of `aiida-quantumespresso-ph`.

Let's get started!

In [6]:
from local_module import load_temp_profile
from aiida.plugins import DataFactory

# If you download this file, you can run it with your own profile.
# Put these lines instead:
# from aiida import load_profile
# load_profile()
load_temp_profile(
    name="dynmat-tutorial",
    add_computer=True,
    add_pw_code=True,
    add_ph_code=True,
    add_sssp=True,
)

StructureData = DataFactory("core.structure")


## Defining a structure

We need to create or load the structure for which to compute the dynamical matrix. This must be a {py:class}`~aiida.orm.StructureData`.
Here, we define the structure of NaCl salt, which has only 2 atoms in its primitive cell.

Let's define the silicon structure using the ASE module:

In [7]:
# Create a silicon cubic crystal
a = 2.716
cell = [[0,a,a],[a,0,a],[a,a,0]]

structure = StructureData(cell=cell)
structure.append_atom(position=(a,a,a), symbols="Si")
structure.append_atom(position=(1.5*a,1.5*a,1.5*a), symbols="Si")

## Define the inputs of the `DynamicalMatrixWorkChain` workflow

Let's define the inputs:

* ``pw_code``: the ``Code`` which will run the `pw.x` binary
* ``ph_code``: the ``Code`` which will run the `ph.x` binary
* ``structure``: the StructureData node containing the information regarding the crystal structure

In [8]:
from aiida.orm import load_code, Dict
from aiida_quantumespresso_ph.workflows.dynamical_matrix import DynamicalMatrixWorkChain

builder = DynamicalMatrixWorkChain.get_builder_from_protocol(
    pw_code=load_code("pw@localhost"),
    ph_code=load_code("ph@localhost"),
    structure=structure,
    protocol="fast",
)

/opt/conda/lib/python3.10/site-packages/aiida_quantumespresso/workflows/protocols/utils.py:147: UserWarning: Found unrecognised key in overrides: parallelize_qpoints
  warnings.warn(f'Found unrecognised key in overrides: {full_key}')
/opt/conda/lib/python3.10/site-packages/aiida_quantumespresso/workflows/protocols/utils.py:147: UserWarning: Found unrecognised key in overrides: ph.qpoints
  warnings.warn(f'Found unrecognised key in overrides: {full_key}')


And now submit the calculation!

In [9]:
from aiida.engine import run_get_node

results, node = run_get_node(builder)

12/19/2025 10:57:38 AM <69238> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [124|DynamicalMatrixWorkChain|setup]: no parent given
12/19/2025 10:57:39 AM <69238> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [124|DynamicalMatrixWorkChain|run_relax]: launching PwRelaxWorkChain<126>
12/19/2025 10:57:39 AM <69238> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [126|PwRelaxWorkChain|run_relax]: launching PwBaseWorkChain<129>
12/19/2025 10:57:39 AM <69238> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [129|PwBaseWorkChain|run_process]: launching PwCalculation<134> iteration #1
/opt/conda/lib/python3.10/site-packages/aiida/orm/nodes/data/code/legacy.py:556: AiidaDeprecationWarning: `Code.is_local` method is deprecated, use a `PortableCode` instance and check the type. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.10/site-packages/aiida/orm/nodes/data/code/legacy.py:517: AiidaDeprecati

## Inspect the outputs

You can now look into the outputs of the workflow.

In [10]:
print("DynamicalMatrixWorkChain is finished ok?", node.is_finished_ok)

DynamicalMatrixWorkChain is finished ok? False
